In [6]:
%run classes.ipynb
# Training cell
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
emptyRatioA = 47
emptyRatioB = 30
emptyRatioC = 24
objectwidth = 83
objectheight = 122
bend = 225

def train(windowSize, windowAdvance, crossvalidate=False, neighbourhoodDistance=[5], neighbourhoodZ=[3], noOfBins=[20], n_neighbors=[3]):

    odtype = 'moran'
    porositythreshold = 0.5
    #windowSize = 75
    #windowAdvance = 25
    #noOfBins = 20
    test_size = 0.4
    hsegments = [0,26,50,74,98,122]
    mask = np.zeros((objectwidth, objectheight))

    X_train = readRawTreeImages()
    labelsA = np.copy(getLabels(porositythreshold))

    # Todo: Uncomment this line

    mask = makeMask(X_train, emptyRatioA)

    bgr = backgroundRemover(mask)
    results = []
    for nd in neighbourhoodDistance:
        for z in neighbourhoodZ:
            oc1 = outlierCalculator(odtype, nd, z)
            # a1 = accumulator(windowSize, windowOffset)
            # Hur modellerar vi det här? Objekt med olika höjd blir kaos, eller? Skulle kunna skicka tomma bilder när de är slut. 

            # settings = [(5, 3),
            #             (15, 8),
            #             (25, 13),
            #             (50, 25),
            #             (75, 30),
            #             (105, 52),
            #             #(125, 15),
            #             #(150, 15),
            #             #(175, 20),
            #             #(200, 10),
            #             (212, 13)]

            allYs = np.array([])
            reallyAllOcVals = []

            # for (windowSize, windowAdvance) in settings:
            ab = blockAccumulator(windowSize, windowAdvance)

            trainObjects1 = np.moveaxis(X_train, 1, 0)

            allOcvals = []
            allPresValues = []
            i = 0
            ocA = []
            ocB = []
            for image in trainObjects1:
                # Ta bilden, omvandla till en array av bilder (en bild per obj)
                # Vilken struktur behöver preprocess? Lista med numpy array
                # image är 26x122x83
                # segments ska vara (26*5)x122x83
                noback = bgr.removeBackground(image)
                segments = makesegmentsA(noback, hsegments)
                segmentblocks = ab.next(segments)
                if(segmentblocks is False):
                    continue
                # plt.hist(trainoutliers[0][1].flatten(), bins=bincount, range=(minval, maxval))
                ocvals = oc1.calculate(segmentblocks)
                ocA.append(ocvals)
                i+=1
                # if(i == 0 or i == 1 or i == 2):
                #     plt.imshow(pres[0])
                #     plt.figure()
                #     plt.hist(pres[0].flatten())
                #     plt.yscale('log')
                #     plt.figure()
                # ocVals: List[layers][objects][matrix]

            acount = len(trainObjects1)
            ab.reset()

            X_train2 = readRawHouseImages()
            trainObjects2 = np.moveaxis(X_train2, 1, 0)
            maskB = makeMaskB(X_train2, emptyRatioB)
            bgrb = backgroundRemover(maskB)
            for image in trainObjects2:
                noback = bgrb.removeBackground(image)
                segments = noback
                segmentblocks = ab.next(segments)
                if(segmentblocks is False):
                    continue
                ocvals = oc1.calculate(segmentblocks)
                ocB.append(ocvals)


            ## Done with oc calculation, let's get some labels ##
            labelRanges = [(i, i+windowSize) for i in range(0, bend-windowSize+1, windowAdvance)]
            ytrainb = [getLabelsB(objIndex, 0.5, start, end) for (start, end) in labelRanges for objIndex in range(1,29)]
            # Äpplen och päron eller? Vad är allOcVals, vad är ytrainb, varför dela med Y_train, varför repeitionsA?
            # Vad är ytrainb som har en längd på 2072?
            # Det är 74 (antalet fönster i ett objekt) x 28 (antalet objekt)
            ytraina = np.tile(labelsA.astype('int'), [len(ocA)])
            ytrainboth = np.concatenate([ytraina, ytrainb])
            
            reallyAllOcVals = ocA + ocB

            ocmin = np.nanmin(np.concatenate([np.concatenate([o.flatten() for o in oo]) for oo in reallyAllOcVals]))
            ocmax = np.nanmax(np.concatenate([np.concatenate([o.flatten() for o in oo]) for oo in reallyAllOcVals]))
            
            for nobs in noOfBins:
                for k in n_neighbors:
                    e1 = encoder(nobs, ocmin, ocmax)

                    allTrainingHists = []
                    for ocvals in reallyAllOcVals:
                        if len(ocvals) > 0:
                            # Print a chart here
                            # tar encode en lista av 3d?
                            hists = e1.encode(ocvals)
                            allTrainingHists.extend(hists)

                    knn = neighbors.KNeighborsClassifier(k, weights="uniform", n_jobs=-1)
                    scaler = StandardScaler()
                    clf = Pipeline([('scaler', scaler), ('classifier', knn)])
                    scores = cross_val_score(clf, allTrainingHists, ytrainboth, cv=5, scoring='roc_auc')
                    clf.fit(allTrainingHists, ytrainboth)
                    results.append((clf, oc1, e1), nd, z, nobs, k, scores.mean())
        # return the best setting
        best = max(results, key=lambda x: x[-1])
        return best

def makesegmentsA(image, hsegments):
        segments = []
        for obimg in image:
            for i in range(len(hsegments)-1):
                segments.append(obimg[hsegments[i]:hsegments[i+1],:])
        return segments

(Pipeline(steps=[('scaler', StandardScaler()),
                 ('classifier', KNeighborsClassifier(n_jobs=-1, n_neighbors=3))]),
 <__main__.encoder at 0x16f40d090>)